In [1]:
import numpy as np
import pandas as pd
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from scipy.stats import spearmanr
import functools as ft
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import gdown

In [2]:
import sys
import gensim
import codecs
import json
from gensim.models.keyedvectors import Word2VecKeyedVectors
from gensim.models import KeyedVectors
import numpy as np
import random
import sklearn
from sklearn import model_selection
from sklearn import cluster
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import DictVectorizer
import numpy as np
from docopt import docopt
import torch
from transformers import *
import pickle
from tqdm import tqdm

import scipy
from scipy import linalg
from scipy import sparse
from scipy.stats.stats import pearsonr
import tqdm
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier, SGDRegressor, Perceptron, LogisticRegression


import warnings
warnings.filterwarnings("ignore")

import pickle
from collections import defaultdict, Counter
from typing import List, Dict

import torch
from torch import utils


import pandas as pd

import time
from mlxtend.math import vectorspace_orthonormalization

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import keras

In [4]:
import torch
from torch import Tensor
from transformers import BertTokenizer, BertModel

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  ).to(device)

loading configuration file config.json from cache at /home/leiding/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/leiding/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt

In [6]:
def encode_text(model, data):
    """
    encode the text
    :param model: encoding model
    :param data: data
    :return: two numpy matrices of the data:
                first: average of all tokens in each sentence
                second: cls token of each sentence
    """
    all_data_avg = []
    batch = []
    count = 0
    for w in data:
        tokens = tokenizer.encode(w, add_special_tokens=False)
        batch.append(tokens)
        input_ids = torch.tensor(batch).to(device)
        with torch.no_grad():
            last_hidden_states = model(input_ids)[0]
            all_data_avg.append((last_hidden_states.squeeze(0).mean(dim=0).cpu()).numpy())
        batch = []
        count = count + 1
        if count%1000 == 0:
            print(count)
    return np.array(all_data_avg)

In [8]:
X_train = pd.read_csv('Train_emb.csv')
X_train = X_train.values[:,1:]

In [9]:
def directions(w1,w2,model):
    
    tokens1 = tokenizer.encode(w1, add_special_tokens=False)
    input_ids1 = torch.tensor([tokens1]).to(device)
    with torch.no_grad():
        last_hidden_states1 = model(input_ids1)[0]
        embs1 = (last_hidden_states1.squeeze(0).mean(dim=0).cpu()).numpy()
    
    tokens2 = tokenizer.encode(w2, add_special_tokens=False)
    input_ids2 = torch.tensor([tokens2]).to(device)
    with torch.no_grad():
        last_hidden_states2 = model(input_ids2)[0]
        embs2 = (last_hidden_states2.squeeze(0).mean(dim=0).cpu()).numpy()
        
    return embs1 - embs2

In [10]:
d1 = directions('black people','white people',model)
d2 = directions('European American','African American',model)
d3 = directions('father','mother',model)

In [11]:
similarity = []

In [12]:
from scipy.spatial.distance import cosine

In [13]:
for i in range(X_train.shape[0]):
    similarity.append(1-cosine(X_train[i,:], d1))
    if i%10000 == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000


In [14]:
df1 = pd.DataFrame(columns = ['word','sim'])

In [15]:
df1['sim'] = similarity

In [16]:
df1 = df1.sort_values(by="sim" , ascending=False)

In [17]:
df1

,word,sim
7734,NaN,0.276604
237618,NaN,0.276604
58291,NaN,0.276604
20643,NaN,0.250199
19759,NaN,0.245253
...,...,...
5945,NaN,-0.340169
293453,NaN,-0.342731
147220,NaN,-0.343024
32699,NaN,-0.350447


In [18]:
X_ = []
Y_ = []

In [19]:
for i in range(75000):
    X_.append(X_train[df1.index[i],:])
    Y_.append(1)
    X_.append(X_train[df1.index[-i-1],:])
    Y_.append(0)

In [20]:
X_train = np.array(X_)
Y_train = np.array(Y_)

In [21]:
clf = LogisticRegression(warm_start = True, penalty = 'l2',
                         solver = "saga", multi_class = 'multinomial', fit_intercept = False,
                         verbose = 5, n_jobs = 90, random_state = 1, max_iter = 7)

        
start = time.time()
clf.fit(X_train, Y_train)
print("time: {}".format(time.time() - start))
print(clf.score(X_train, Y_train))

[Parallel(n_jobs=90)]: Using backend ThreadingBackend with 90 concurrent workers.


Epoch 1, change: 1.00000000
Epoch 2, change: 0.05874477
Epoch 3, change: 0.01846461
Epoch 4, change: 0.01249123
Epoch 5, change: 0.01127643
Epoch 6, change: 0.00602118
max_iter reached after 12 seconds
time: 11.430743217468262
Epoch 7, change: 0.00501641
1.0


[Parallel(n_jobs=90)]: Done   1 out of   1 | elapsed:   11.3s finished


In [22]:
Y = clf.predict_proba(X_train)

In [23]:
def slice_freq(Y, interval):
    y_f = pd.Series(Y)
    a = pd.cut(y_f,interval)
    b=a.value_counts()
    frequency = b.sort_index().values
    return frequency

In [24]:
def SDR_multi(X_train, Y, K, h):
    X = X_train
    n = X.shape[0]
    d_x = X.shape[1]
    d_y = Y.shape[1]

    interval = np.linspace(0, 1, h+1,endpoint=True)
    Gamma_PMS = np.zeros([d_x,d_x])
    for i in range(d_y):  
        freq = slice_freq(Y[:,i], interval)
        Gamma = np.zeros([d_x,d_x])
        for j in range(h):
            ph = freq[j]/n
            index = np.where((Y[:,i]>=interval[j])&(Y[:,i]<=interval[j+1]))
            mh = (1/(n*ph))*np.sum(X[index,:],0)
            Gamma = Gamma + ph*np.dot(mh.T,mh)
        la_g, v_g = np.linalg.eig(Gamma)
        la_g = la_g.real
        Gamma_PMS = Gamma_PMS + la_g[0]*Gamma
    la_PMS, v_PMS = np.linalg.eig(Gamma_PMS)
    v_PMS = v_PMS.real
    beta1 = vectorspace_orthonormalization(v_PMS[:,:K])
    #beta1 = v_PMS[:,:K]
    return np.dot(beta1,beta1.T), np.eye(d_x)-np.dot(beta1,beta1.T) 

In [25]:
def iteration_estimation_v2(X_train, Y, Iter, q, h):
    N_sample = len(X_train)
    d = X_train.shape[1]
    beta1 = np.eye(d)
    
    X_batch = X_train
    n = X_batch.shape[0]
    X = X_batch-np.mean(X_batch,0).reshape(1,d)
    Cx = np.dot(X_batch.T,X_batch)/n
    la, v = np.linalg.eig(Cx)
    la = la.real
    v = v.real
    Cx12 = np.dot(np.dot(v,np.diag(la**(0.5))),v.T)
    X_train = np.dot(X_batch,np.linalg.pinv(Cx12,rcond=1e-8))
    beta1, beta2 = SDR_multi(X_train, Y, q, h)
    print(np.diag(beta1)[:5])
    if Iter == 0:
        return beta1, beta2
    else:
        for j in range(Iter):
            idx = np.random.rand(X_train.shape[0]) < 1
            X_train = np.dot(X_train,beta1)
            beta1, beta2 = SDR_multi(X_train[idx], Y[idx], q, h)
            print(np.diag(beta1)[:5])
    return beta1, beta2

In [47]:
beta1, beta2 = iteration_estimation_v2(X_train, Y, 2, 600, 100)

[0.79412697 0.79035232 0.77563328 0.74640559 0.80392577]
[0.79451847 0.79574049 0.77495668 0.74808496 0.80328657]
[0.79582952 0.78765717 0.77430954 0.74689911 0.80065451]


In [48]:
import pandas as pd

In [49]:
import pdb

In [50]:
import json
import os
import random
import re

import numpy as np
import torch
import weat


class SEATRunner:
    """Runs SEAT tests for a given HuggingFace transformers model.
    Implementation taken from: https://github.com/W4ngatang/sent-bias.
    """

    # Extension for files containing SEAT tests.
    TEST_EXT = ".jsonl"

    def __init__(
        self,
        model,
        tokenizer,
        tests,
        data_dir,
        experiment_id,
        n_samples=100000,
        parametric=False,
        seed=0,
    ):
        """Initializes a SEAT test runner.
        Args:
            model: HuggingFace model (e.g., BertModel) to evaluate.
            tokenizer: HuggingFace tokenizer (e.g., BertTokenizer) used for pre-processing.
            tests (`str`): Comma separated list of SEAT tests to run. SEAT test files should
                be in `data_dir` and have corresponding names with extension ".jsonl".
            data_dir (`str`): Path to directory containing the SEAT tests.
            experiment_id (`str`): Experiment identifier. Used for logging.
            n_samples (`int`): Number of permutation test samples used when estimating p-values
                (exact test is used if there are fewer than this many permutations).
            parametric (`bool`): Use parametric test (normal assumption) to compute p-values.
            seed (`int`): Random seed.
        """
        self._model = model
        self._tokenizer = tokenizer
        self._tests = tests
        self._data_dir = data_dir
        self._experiment_id = experiment_id
        self._n_samples = n_samples
        self._parametric = parametric
        self._seed = seed

    def __call__(self):
        """Runs specified SEAT tests.
        Returns:
            `list` of `dict`s containing the SEAT test results.
        """
        random.seed(self._seed)
        np.random.seed(self._seed)

        all_tests = sorted(
            [
                entry[: -len(self.TEST_EXT)]
                for entry in os.listdir(self._data_dir)
                if not entry.startswith(".") and entry.endswith(self.TEST_EXT)
            ],
            key=_test_sort_key,
        )

        # Use the specified tests, otherwise, run all SEAT tests.
        tests = self._tests or all_tests

        results = []
        for test in tests:
            print(f"Running test {test}")

            # Load the test data.
            encs = _load_json(os.path.join(self._data_dir, f"{test}{self.TEST_EXT}"))

            print("Computing sentence encodings")
            encs_targ1 = _encode(
                self._model, self._tokenizer, encs["targ1"]["examples"]
            )
            encs_targ2 = _encode(
                self._model, self._tokenizer, encs["targ2"]["examples"]
            )
            encs_attr1 = _encode(
                self._model, self._tokenizer, encs["attr1"]["examples"]
            )
            encs_attr2 = _encode(
                self._model, self._tokenizer, encs["attr2"]["examples"]
            )

            encs["targ1"]["encs"] = encs_targ1
            encs["targ2"]["encs"] = encs_targ2
            encs["attr1"]["encs"] = encs_attr1
            encs["attr2"]["encs"] = encs_attr2

            print("\tDone!")

            # Run the test on the encodings.
            esize, pval = weat.run_test(
                encs, n_samples=self._n_samples, parametric=self._parametric
            )

            results.append(
                {
                    "experiment_id": self._experiment_id,
                    "test": test,
                    "p_value": pval,
                    "effect_size": esize,
                }
            )

        return results


def _test_sort_key(test):
    """Return tuple to be used as a sort key for the specified test name.
    Break test name into pieces consisting of the integers in the name
    and the strings in between them.
    """
    key = ()
    prev_end = 0
    for match in re.finditer(r"\d+", test):
        key = key + (test[prev_end : match.start()], int(match.group(0)))
        prev_end = match.end()
    key = key + (test[prev_end:],)

    return key


def _split_comma_and_check(arg_str, allowed_set, item_type):
    """Given a comma-separated string of items, split on commas and check if
    all items are in allowed_set -- item_type is just for the assert message.
    """
    items = arg_str.split(",")
    for item in items:
        if item not in allowed_set:
            raise ValueError(f"Unknown {item_type}: {item}!")
    return items


def _load_json(sent_file):
    """Load from json. We expect a certain format later, so do some post processing."""
    print(f"Loading {sent_file}...")
    all_data = json.load(open(sent_file, "r"))
    data = {}
    for k, v in all_data.items():
        examples = v["examples"]
        data[k] = examples
        v["examples"] = examples

    return all_data


def _encode(model, tokenizer, texts):
    encs = {}
    for text in texts:
        # Encode each example.
        inputs = tokenizer(text, return_tensors="pt")
        outputs = model(**inputs)

        # Average over the last layer of hidden representations.
        enc = outputs["last_hidden_state"]
        enc = enc.mean(dim=1)
        
        

        # Following May et al., normalize the representation.
        encs[text] = enc.detach().view(-1).numpy()
        encs[text] = proj.dot(encs[text])
        encs[text] /= np.linalg.norm(encs[text])
        #encs[text] = proj.dot(encs[text]) 
        

    return encs

In [51]:
import glob
paths = glob.glob('./Other_seat_test/Race/*.jsonl')
paths.sort()

In [52]:
device = 'cpu'

In [53]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  ).to(device)

loading configuration file config.json from cache at /home/leiding/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/leiding/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/p

In [54]:
proj = beta2

In [55]:
results = []
for test in paths:
    print(f"Running test {test}")

    # Load the test data.
    encs = _load_json(test)

    print("Computing sentence encodings")
    encs_targ1 = _encode(
        model, tokenizer, encs["targ1"]["examples"]
    )
    encs_targ2 = _encode(
        model, tokenizer, encs["targ2"]["examples"]
    )
    encs_attr1 = _encode(
        model, tokenizer, encs["attr1"]["examples"]
    )
    encs_attr2 = _encode(
        model, tokenizer, encs["attr2"]["examples"]
    )

    encs["targ1"]["encs"] = encs_targ1
    encs["targ2"]["encs"] = encs_targ2
    encs["attr1"]["encs"] = encs_attr1
    encs["attr2"]["encs"] = encs_attr2

    print("\tDone!")

    # Run the test on the encodings.
    esize, pval = weat.run_test(
        encs, n_samples=100000, parametric=False
    )

    results.append(
        {
            "test": test,
            "p_value": pval,
            "effect_size": esize,
        }
    )


Running test ./Other_seat_test/Race/sent-angry_black_woman_stereotype.jsonl
Loading ./Other_seat_test/Race/sent-angry_black_woman_stereotype.jsonl...
Computing sentence encodings
	Done!
Computing cosine similarities...
Null hypothesis: no difference between WhiteFemaleNames and BlackFemaleNames in association to attributes NearAntonyms and AngryBlackWomanStereotype
Computing pval...
Using non-parametric test
Drawing 99999 samples (and biasing by 1)
pval: 0.440
computing effect size...
esize: 0.020
Running test ./Other_seat_test/Race/sent-angry_black_woman_stereotype_b.jsonl
Loading ./Other_seat_test/Race/sent-angry_black_woman_stereotype_b.jsonl...
Computing sentence encodings
	Done!
Computing cosine similarities...
Null hypothesis: no difference between WhiteFemaleTerms and BlackFemaleTerms in association to attributes NearAntonyms and AngryBlackWomanStereotype
Computing pval...
Using non-parametric test
Drawing 99999 samples (and biasing by 1)
pval: 0.014
computing effect size...
esi

In [56]:
results

[{'test': './Other_seat_test/Race/sent-angry_black_woman_stereotype.jsonl',
  'p_value': 0.43967,
  'effect_size': 0.019651265977989975},
 {'test': './Other_seat_test/Race/sent-angry_black_woman_stereotype_b.jsonl',
  'p_value': 0.01424,
  'effect_size': 0.42828790867751976},
 {'test': './Other_seat_test/Race/sent-weat3.jsonl',
  'p_value': 1e-05,
  'effect_size': 0.5425307841592961},
 {'test': './Other_seat_test/Race/sent-weat3b.jsonl',
  'p_value': 0.02887,
  'effect_size': 0.19270187350664095},
 {'test': './Other_seat_test/Race/sent-weat4.jsonl',
  'p_value': 1e-05,
  'effect_size': 0.6113100600813963},
 {'test': './Other_seat_test/Race/sent-weat5.jsonl',
  'p_value': 1e-05,
  'effect_size': 0.7160457958048247},
 {'test': './Other_seat_test/Race/sent-weat5b.jsonl',
  'p_value': 1e-05,
  'effect_size': 0.5142894570272065}]

In [57]:
effect_size = []
ss = 0
for value in results:
    v = value['effect_size']
    effect_size.append(v)
    ss += abs(v)
print(ss/len(effect_size))

0.43211673503355347


In [58]:
effect_size

[0.019651265977989975,
 0.42828790867751976,
 0.5425307841592961,
 0.19270187350664095,
 0.6113100600813963,
 0.7160457958048247,
 0.5142894570272065]